# Tutoriel d'usage de l'API

In [1]:
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

In [2]:
import numpy as np 

In [3]:
http://51.91.251.0:3000/remaining-requests/12e1e6fc-456b-4738-ba59-b90c3386f583

SyntaxError: invalid syntax (<ipython-input-3-3b400d8b66f7>, line 1)

## Creating an avatar

The admin of the competition can create users. A user can then create as many avatars as they wish.

In [134]:
import requests

user_id = '12e1e6fc-456b-4738-ba59-b90c3386f583'
name = 'ema-lithuanian1-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [200]>

In [91]:
# name = 'max-second-avatar'
# r = requests.post(path(f"avatars/{user_id}/{name}"))
# r

In [135]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6217 ema-first-avatar
34250 ema-austrian1-avatar
34256 ema-belgian1-avatar
34257 ema-bulgarian1-avatar
34258 ema-croatian1-avatar
34305 ema-german1-avatar
34259 ema-cypriot1-avatar
34260 ema-czech1-avatar
34264 ema-finnish1-avatar
34265 ema-french1-avatar
34261 ema-danish1-avatar
34262 ema-dutch1-avatar
34263 ema-estanian1-avatar
34429 ema-greek1-avatar
34457 ema-irish1-avatar
34455 ema-hungarian2-avatar
34456 ema-hungarian3-avatar
34458 ema-italian1-avatar
35123 ema-belgian3-avatar
37849 ema-luxembourgish1-avatar
38438 ema-luxembourgish2-avatar
73369 ema-maltese1-avatar
73371 ema-polish2-avatar
73370 ema-polish1-avatar
92219 ema-irish2-avatar
92220 ema-italian2-avatar
92221 ema-latvian1-avatar
92222 ema-lithuanian1-avatar


## Making pricing requests

In [93]:
language_list=['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish', 'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish' ,'swedish']

city_list=['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna','vilnius']

date_list=list(np.arange(1,45,1))

In [94]:
## A modifier logiquement selon dates souhaitées

date_list_soon=[2,4,5]

date_list_middle=[9,11,13,14]

date_list_away=[35,37,40,41]



In [149]:
language_test='lithuanian'
avatar_test='ema-lithuanian1-avatar'
date_test=date_list_middle

In [150]:
print(avatar_test, language_test,date_test)

ema-lithuanian1-avatar lithuanian [9, 11, 13, 14]


In [151]:
requests_list=[]

for date in reversed(date_test) : 
    for city in city_list : 
        params = {
            "avatar_name": avatar_test,
            "language": language_test,
            "city": city,
            "date": date,
            "mobile": np.random.randint(0,2),
        }
        r = requests.get(path(f"pricing/{user_id}"), params=params)
        r.json()
        
        requests_list.append(r)
       

## Build a training set

In [152]:
import pandas as pd

pricing_requests = []

for r in requests_list:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,750,116,10,amsterdam,14,lithuanian,0,92222
1,529,247,20,amsterdam,14,lithuanian,0,92222
2,193,153,9,amsterdam,14,lithuanian,0,92222
3,760,229,14,amsterdam,14,lithuanian,0,92222
4,212,244,12,amsterdam,14,lithuanian,0,92222


In [153]:
len(pricing_requests)

3952

In [154]:
# print(pricing_requests[ pricing_requests['mobile'] == 0])

In [155]:
data=pd.read_csv('./Datatrain/test_requete.csv')

In [156]:
data=data.drop(['Unnamed: 0'],axis=1)

In [157]:
pricing_requests=pd.concat([data,pricing_requests])

In [158]:
pricing_requests

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,556,175,60,copenhagen,30,hungarian,0,6217
1,664,284,111,copenhagen,30,hungarian,0,6217
2,96,124,37,copenhagen,30,hungarian,0,6217
3,865,277,107,copenhagen,30,hungarian,0,6217
4,58,203,65,copenhagen,30,hungarian,0,6217
...,...,...,...,...,...,...,...,...
88,654,227,6,vilnius,9,lithuanian,1,92222
89,881,188,8,vilnius,9,lithuanian,1,92222
90,221,59,1,vilnius,9,lithuanian,1,92222
91,887,136,8,vilnius,9,lithuanian,1,92222


In [159]:
pricing_requests.to_csv('./Datatrain/test_requete.csv')

In [160]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,556,175,60,copenhagen,30,hungarian,0,6217,Boss Western,J.Halliday Inn,1,0,0
1,664,284,111,copenhagen,30,hungarian,0,6217,Accar Hotels,Safitel,0,1,0
2,96,124,37,copenhagen,30,hungarian,0,6217,Boss Western,Boss Western,0,0,0
3,865,277,107,copenhagen,30,hungarian,0,6217,Chillton Worldwide,Chill Garden Inn,1,1,0
4,58,203,65,copenhagen,30,hungarian,0,6217,Yin Yang,Royal Lotus,1,0,2


In [161]:
pricing_requests.to_csv('./Datatrain/test_requete_features.csv')

In [116]:
# sample_submission = pd.read_csv('../input/defi-ia-2023/sample_submission.csv')
# sample_submission.head()

In [117]:
# sample_submission.to_csv('submission.csv', index=False)